<a href="https://colab.research.google.com/github/softip/projeto_IA368X/blob/main/notebooks/Tratamentode_Dados_para_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import pandas as pd

#importa dataset do github
df = []
for i in range(1,10):
  df.append(pd.read_csv("https://raw.githubusercontent.com/softip/projeto_IA368X/main/data/external/dados_covid_ce_{}.csv".format(i)))

dfTotal = pd.DataFrame()
dfTotal = pd.concat(df)
dfTotal.reset_index()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35,36,37,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,24,25,26,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result

,index,codigoPaciente,paisPaciente,estadoPaciente,codigoMunicipioPaciente,municipioPaciente,sexoPaciente,idadePaciente,dataNotificacao,dataColetaExame,dataResultadoExame,resultadoFinalExame,dataInicioSintomas,tipoTesteExame,racaCorPaciente,municipioNotificacaoEsus,profissionalSaude,cnesNotificacaoEsus,idSivep,classificacaoEstadoSivep,dataInternacaoSivep,dataEntradaUtisSvep,bairroPaciente,comorbidadeNeurologiaSivep,dataSolicitacaoExame,obitoConfirmado,dataObito,tipoObito,dataNotificacaoObito,comorbidadeCardiovascularSivep,comorbidadeObesidadeSivep,evolucaoCaso,dataEvolucaoCasoSivep,comorbidadeHematologiaSivep,comorbidadeSindromeDownSivep,comorbidadeAsmaSivep,comorbidadeDiabetesSivep,comorbidadeImunodeficienciaSivep,comorbidadeRenalSivep,comorbidadePuerperaSivep,comorbidadePneumopatiaSivep,dataSaidaUtisSvep
0,0,579fce2867018c510bdfbc9c64c14a34,BRASIL,CE,230490.0,GROAIRAS,FEMININO,25.0,2020-01-01T03:00:00.000+0000,2021-01-08T03:00:00.000+0000,2021-01-08T03:00:00.000+0000,Negativo,2020-01-01T03:00:00.000+0000,Teste Rápido - Anticorpo,Parda,GROAIRAS,0.0,6565239,3.159248e+11,Não realizado.0,2020-06-18T03:00:00.000+0000,2020-06-18T03:00:00.000+0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,504dc6479d137aef7556a9878626aef8,NaN,CE,230240.0,BOA VIAGEM,MASCULINO,39.0,2020-01-01T03:00:00.000+0000,2020-10-20T03:00:00.000+0000,2020-10-20T03:00:00.000+0000,Positivo,2020-01-01T03:00:00.000+0000,Teste Rápido - Anticorpo,Parda,MARACANAU,0.0,NaN,NaN,NaN,NaN,NaN,ALTO DO MOTOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,9b1fca71df2ed7f6bcbc9f20122c78d2,BRASIL,CE,230075.0,AMONTADA,MASCULINO,66.0,2020-01-02T03:00:00.000+0000,2021-01-02T03:00:00.000+0000,2021-01-02T03:00:00.000+0000,Negativo,2020-01-02T03:00:00.000+0000,Teste Rápido - Anticorpo,Parda,AMONTADA,0.0,6460038,3.161099e+11,Negativo.0,2021-01-02T03:00:00.000+0000,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,fd948fcaca62c97a2f57a1717e7ad128,NaN,CE,230170.0,AURORA,MASCULINO,57.0,2020-01-02T03:00:00.000+0000,2020-01-02T03:00:00.000+0000,2020-01-02T03:00:00.000+0000,Positivo,2020-01-02T03:00:00.000+0000,Teste Rápido - Antígeno,Parda,PRINCESA ISABEL,0.0,NaN,NaN,NaN,NaN,NaN,AURORA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,fdccba56299c30f73a717ec52e5fc740,NaN,CE,231030.0,PARAMBU,FEMININO,25.0,2020-01-04T03:00:00.000+0000,2021-01-08T03:00:00.000+0000,2021-01-08T03:00:00.000+0000,Positivo,2020-01-01T03:00:00.000+0000,RT-PCR,Parda,PARAMBU,0.0,6362869,NaN,NaN,NaN,NaN,CAIXA D`AGUA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873578,167232,eefa5ea801b53366cce43b2b09a58bdf,NaN,CE,230440.0,FORTALEZA,FEMININO,3.0,2021-05-21T03:00:00.000+0000,NaN,NaN,NaN,2021-05-15T03:00:00.000+0000,NaN,Parda,FORTALEZA,0.0,407836,NaN,NaN,NaN,NaN,PICI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1873579,167233,ab18e9b4902269f89d864e17da48e35f,NaN,CE,231290.0,SOBRAL,FEMININO,16.0,2021-05-19T03:00:00.000+0000,NaN,NaN,NaN,2021-05-17T03:00:00.000+0000,NaN,Parda,SOBRAL,1.0,2.42412e+06,NaN,NaN,NaN,NaN,PEDRINHAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1873580,167234,ad2979dd853f247793d4ec2fb967be3b,BRASIL,CE,230440.0,FORTALEZA,MASCULINO,43.0,2021-05-06T03:00:00.000+0000,2021-04-26T03:00:00.000+0000,2021-04-28T04:26:39.000+0000,Positivo,2021-05-04T03:00:00.000+0000,RT-PCR,Parda,FORTALEZA,0.0,7.99322e+06,3.162033e+11,Negativo.0,NaN,2021-05-06T03:00:00.000+0000,SIQUEIRA,NaN,2021-04-26T03:00:00.000+0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN
1873581,167235,114849ef4d40d96f060316c8be22abf3,NaN,CE,230370.0,CAUCAIA,MASCULINO,33.0,2021-05-09T03:00:00.000+0000,NaN,NaN,NaN,2021-05-07T03:00:0

In [125]:
#mostra os campos e a quantidade de dados faltantes
dfTotal.isna().sum()

codigoPaciente                            0
paisPaciente                        1786289
estadoPaciente                          256
codigoMunicipioPaciente               14685
municipioPaciente                        40
sexoPaciente                             52
idadePaciente                            80
dataNotificacao                           0
dataColetaExame                      375130
dataResultadoExame                   379586
resultadoFinalExame                  117663
dataInicioSintomas                    17533
tipoTesteExame                       117983
racaCorPaciente                       95777
municipioNotificacaoEsus              30541
profissionalSaude                     30503
cnesNotificacaoEsus                  385965
idSivep                             1791914
classificacaoEstadoSivep            1805751
dataInternacaoSivep                 1801016
dataEntradaUtisSvep                 1855513
bairroPaciente                        19590
comorbidadeNeurologiaSivep      

In [126]:
#mostra a quantidade de registros no dataset
dfTotal.shape

(1873583, 41)

**Seleciona apenas as colunas de interesse**

In [127]:
#Define as colunas de interesse
features = ["codigoPaciente",
"codigoMunicipioPaciente",
"sexoPaciente",
"idadePaciente",
"racaCorPaciente",
"comorbidadeNeurologiaSivep",
"comorbidadeCardiovascularSivep",
"comorbidadeObesidadeSivep",
"comorbidadeHematologiaSivep",
"comorbidadeSindromeDownSivep",
"comorbidadeAsmaSivep",
"comorbidadeDiabetesSivep",
"comorbidadeImunodeficienciaSivep",
"comorbidadeRenalSivep",
"comorbidadePuerperaSivep",
"comorbidadePneumopatiaSivep",
"dataNotificacao",
"resultadoFinalExame",
"dataInternacaoSivep",
"dataEntradaUtisSvep",
"obitoConfirmado",
"dataObito",
"tipoObito"]

#seleciona apenas as colunas de interesse
dfTotalColunasEspecificas = dfTotal.loc[:,features]

**Segundo Filtro: Selecionar apenas os pacientes que vieram a óbito**

In [128]:
dfObitosGeral = dfTotalColunasEspecificas.loc[dfTotalColunasEspecificas.dataObito.notna()]

In [129]:
#mostra o número total de registros
dfObitosGeral.shape

(24914, 23)

**Remove do Dataset os óbitos que ainda estão em investigação**

In [130]:
#remove os óbitos em investigação
dfObitosConfirmados = dfObitosGeral.loc[~dfObitosGeral.tipoObito.eq("Em investigação")]


In [131]:
#mostra o número de registros após a filtragem
dfObitosConfirmados.shape

(23896, 23)

**Remove os pacientes duplicados**

In [174]:
#Remove as linhas duplicadas
dfObitosNonLinesDuplicates = dfObitosConfirmados.drop_duplicates(subset=["codigoPaciente"], keep="last")

In [156]:
#mostra o número de registros após a filtragem
dfObitosNonLinesDuplicates.shape

(20122, 23)

**Preenche as campos vázios das combormidades com Não**

In [ ]:
#seleciona apenas as features que possuem comorbidades
comorbidade_cols = [col for col in dfObitosNonLinesDuplicates.columns if 'comorbidade' in col]

#Preenche as comorbidades com valores vázios como se não tivessem a comorbidades
dfObitosNonLinesDuplicates.loc[:,comorbidade_cols] = dfObitosNonLinesDuplicates.loc[:, comorbidade_cols].fillna("Não")

**Preenche a coluna obitoConfirmado com valor  Não se o valor for vázio. A coluna obitoConfirmado indica se a pessoa morreu por Covid. Os valores nulos para essa coluna correspondem aos casos descartados (tipoObito).**

In [176]:
#Existe 1616 obitos que foram descartados e estão com valor nulo na coluna obitoConfirmado
dfObitosNonLinesDuplicates[dfObitosNonLinesDuplicates.tipoObito.eq("Descartado")].obitoConfirmado.value_counts(dropna=False)

NaN    1616
Name: obitoConfirmado, dtype: int64

In [177]:
#Preenche com 0 (Não) a coluna obitoConfirmado se ela estiver vázia
dfObitosNonLinesDuplicates.obitoConfirmado.fillna(0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


**Remove o único paciente sem idade**

In [ ]:
dfObitosNonLinesDuplicates.dropna(subset=["idadePaciente"], inplace=True)

**Remove os pacientes sem código de Municipio - provavelmente não são do Ceará**

In [ ]:
dfObitosNonLinesDuplicates.dropna(subset=["codigoMunicipioPaciente"], inplace=True)

**Verifica a quantidade de registros faltantes**

In [180]:
dfObitosNonLinesDuplicates.isna().sum()

codigoPaciente                          0
codigoMunicipioPaciente                 0
sexoPaciente                            0
idadePaciente                           0
racaCorPaciente                      6173
comorbidadeNeurologiaSivep              0
comorbidadeCardiovascularSivep          0
comorbidadeObesidadeSivep               0
comorbidadeHematologiaSivep             0
comorbidadeSindromeDownSivep            0
comorbidadeAsmaSivep                    0
comorbidadeDiabetesSivep                0
comorbidadeImunodeficienciaSivep        0
comorbidadeRenalSivep                   0
comorbidadePuerperaSivep                0
comorbidadePneumopatiaSivep             0
dataNotificacao                         0
resultadoFinalExame                     0
dataInternacaoSivep                  4540
dataEntradaUtisSvep                 12806
obitoConfirmado                         0
dataObito                               0
tipoObito                               0
dtype: int64

**Altera os valores ignorados nas colunas comorbidades para Não**

In [ ]:
for c in comorbidade_cols:
  dfObitosNonLinesDuplicates[c] = dfObitosNonLinesDuplicates[c].apply(lambda value: "Não" if value == "Ignorado" else value)


**Verifica os valores únicos para todas colunas**

In [192]:
colunas = comorbidade_cols + ["sexoPaciente", "idadePaciente", "racaCorPaciente", "obitoConfirmado", "tipoObito"]
for c in colunas:
  print(c, ":", dfObitosNonLinesDuplicates[c].unique())

comorbidadeNeurologiaSivep : ['Não' 'Sim']
comorbidadeCardiovascularSivep : ['Sim' 'Não']
comorbidadeObesidadeSivep : ['Sim' 'Não']
comorbidadeHematologiaSivep : ['Não' 'Sim']
comorbidadeSindromeDownSivep : ['Não' 'Sim']
comorbidadeAsmaSivep : ['Não' 'Sim']
comorbidadeDiabetesSivep : ['Não' 'Sim']
comorbidadeImunodeficienciaSivep : ['Não' 'Sim']
comorbidadeRenalSivep : ['Não' 'Sim']
comorbidadePuerperaSivep : ['Não' 'Sim']
comorbidadePneumopatiaSivep : ['Não' 'Sim']
sexoPaciente : ['MASCULINO' 'FEMININO']
idadePaciente : [ 68.  32.  73.  85.  36.  72.  56.  57.  71.  66.  61.  41.  84.  96.
  94.  76.  80.  59.  55.  74.  91.  37.  83.  64.  60.  97.  86.  77.
  98.  53.  89.  40.  58.  70.  48.  47.   1.  62.  88.  93.  75.  63.
  90.  69.  39.  42.  92.  87.  81.  95.  82.  78.  67.  43.   0.  30.
  51.  24.  65.  31.  17.  79.  52.  38.  46.  50.  44.  45.  54.  29.
  33.  25.  49.  23.  35.  99. 101. 100.  28.  10.   2.  26.  22.  21.
  19.   5.   3. 102.  16.  34. 104.  18.  15.  

**Agrupa a idade em categorias de 5 anos**

In [ ]:
bins = [x for x in range(0,110, 10)]
labels = ["{} a {}".format(x, x+9) for x in range(0,100, 10)]
labels[-1] = 'mais de 90'
dfObitosNonLinesDuplicates['idadeCategoria'] = pd.cut(dfObitosNonLinesDuplicates['idadePaciente'], bins=bins, labels=labels)

**Mostra a distribuição dos dados**

In [206]:
dfObitosNonLinesDuplicates.obitoConfirmado.value_counts()

1.0    18472
0.0     1615
Name: obitoConfirmado, dtype: int64

**Salva o dataset**

In [207]:
dfObitosNonLinesDuplicates.to_csv("DatasetCovidTratado.csv")